In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [10]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [11]:
%%capture pwd
!pwd

In [12]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('has-furlough-prevented-household-financial-distress-during-the-pandemic',
 300,
 500)

In [87]:
lockdowns = [
    {"start": "2020-03-26", "end": "2020-06-23", "event": "lockdown"},
    {"start": "2020-11-05", "end": "2020-12-02", "event": "lockdown"},
    {"start": "2021-01-06", "end": "2021-04-12", "event": "lockdown"},
]
color_dict = {"covid": "#F3F4D8", "lockdown": "#E2E2E2"}
dl = pd.DataFrame(lockdowns)
dl["color"] = [color_dict[i] for i in dl["event"]]

In [88]:
recessions = [
    {"start": "1990-01-01", "end": "1993-01-01", "event": "recession"},
    {"start": "2008-01-01", "end": "2011-01-01", "event": "recession"},
    {"start": "2020-01-01", "end": "2021-04-01", "event": "covid"},
]
color_dict = {"covid": "#F3F4D8", "recession": "#E2E2E2"}
dr = pd.DataFrame(recessions)
dr["color"] = [color_dict[i] for i in dr["event"]]

# Fig 1

In [41]:
df = pd.read_excel("raw/data figure 1.xlsx")
df.columns = ["date", "furloughed", "lockdown"]

In [42]:
f = "fig1_furloughed"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,furloughed,lockdown
0,2020-03-01,20900,0
1,2020-03-02,25200,0
2,2020-03-03,25300,0
3,2020-03-04,25400,0
4,2020-03-05,25500,0


In [90]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="end",
            titleY=-15,
            labelFontSize=11,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y(
        "furloughed:Q",
        title="",
        axis=alt.Axis(
            title="employees furloughed (millions)",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=75,
            titleY=28,
            titleAngle=0,
            titleFontSize=11,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            format="~s",
        ),
    ),
)
line = base.mark_line(color=colors["eco-turquiose"])
rect = (
    alt.Chart(dl)
    .mark_rect(blend="darken")
    .encode(
        x="start:T",
        x2="end:T",
        color=alt.Color("color:N", scale=None),
    )
)
layer1 = (
    (line + rect)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [92]:
df = pd.read_excel("raw/data figure 2.xlsx", usecols="A:B")
df.columns = ["reduction", "default"]

In [93]:
f = "fig2_default"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,reduction,default
0,0.414446,2.814376
1,0.509338,2.812911
2,0.564972,2.812020
3,0.735294,2.809425
4,0.776532,2.808694


In [130]:
base = (
    alt.Chart(f2)
    .encode(
        x=alt.X(
            "x:Q",
            title="",
            axis=alt.Axis(
                grid=False,
                title="decline in income due to furlough",
                titleAnchor="end",
                titleY=-15,
                labelFontSize=11,
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        y=alt.Y(
            "y:Q",
            title="",
            axis=alt.Axis(
                title="probability of late payment",
                domain=False,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=30,
                titleY=5,
                titleAngle=0,
                titleFontSize=11,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                format=".0%",
            ),
        ),
    )
    .transform_calculate(x="datum.reduction/100")
    .transform_calculate(y="datum.default/100")
)
line = base.mark_line(color=colors["eco-dot"])
rule = (
    base.mark_rule(color=colors["eco-mid-blue"], opacity=0.5)
    .encode(x="r:Q")
    .transform_calculate(r="0.17")
)
text = (
    base.mark_text(color=colors["eco-mid-blue"],align='left')
    .encode(x="rx:Q", y="ry:Q", text="rt:N")
    .transform_calculate(rx="0.19")
    .transform_calculate(ry="0.21")
    .transform_calculate(rt='"Average decline in income"')
    .transform_filter('datum.reduction<0.42')
)
layer1 = (
    (line + rule + text)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)